In [4]:
import os
os.environ['PYSPARK_PYTHON'] = 'python'

In [9]:
import pandas as pd
import numpy as np
from pyspark.sql import *
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.sql.functions import col,desc
from matplotlib import pyplot as plt
import seaborn as sns

In [11]:
from scipy import stats


In [12]:
spark = SparkSession.builder.appName("worksheet significance").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/12/23 11:42:19 WARN Utils: Your hostname, Eshikas-MacBook-Air.local, resolves to a loopback address: 127.0.0.1; using 10.1.12.249 instead (on interface en0)
25/12/23 11:42:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/23 11:42:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [29]:
state_df = spark.read.csv('state.csv' , header = True, inferSchema = True)
state_df.show

<bound method DataFrame.show of DataFrame[Population: int, Income: int, Illiteracy: double, Life Exp: double, Murder: double, HS Grad: double, Frost: int, Area: int]>

In [30]:
state_pd = state_df.toPandas()
state_pd.head()

,Population,Income,Illiteracy,Life Exp,Murder,HS Grad,Frost,Area
0,3615,3624,2.1,69.05,15.1,41.3,20,50708
1,365,6315,1.5,69.31,11.3,66.7,152,566432
2,2212,4530,1.8,70.55,7.8,58.1,15,113417
3,2110,3378,1.9,70.66,10.1,39.9,65,51945
4,21198,5114,1.1,71.71,10.3,62.6,20,156361


In [26]:
state_pd.shape

(2397, 1)

In [31]:
## Hypothesis Testing
state_pd.describe

<bound method NDFrame.describe of     Population  Income  Illiteracy  Life Exp  Murder  HS Grad  Frost    Area
0         3615    3624         2.1     69.05    15.1     41.3     20   50708
1          365    6315         1.5     69.31    11.3     66.7    152  566432
2         2212    4530         1.8     70.55     7.8     58.1     15  113417
3         2110    3378         1.9     70.66    10.1     39.9     65   51945
4        21198    5114         1.1     71.71    10.3     62.6     20  156361
5         2541    4884         0.7     72.06     6.8     63.9    166  103766
6         3100    5348         1.1     72.48     3.1     56.0    139    4862
7          579    4809         0.9     70.06     6.2     54.6    103    1982
8         8277    4815         1.3     70.66    10.7     52.6     11   54090
9         4931    4091         2.0     68.54    13.9     40.6     60   58073
10         868    4963         1.9     73.60     6.2     61.9      0    6425
11         813    4119         0.6     71.

In [32]:
# #test wheather the mean murder rate is 8.5
murder_data = state_pd['Murder'].values
print(murder_data)

[15.1 11.3  7.8 10.1 10.3  6.8  3.1  6.2 10.7 13.9  6.2  5.3 10.3  7.1
  2.3  4.5 10.6 13.2  2.7  8.5  3.3 11.1  2.3 12.5  9.3  5.   2.9 11.5
  3.3  5.2  9.7 10.9 11.1  1.4  7.4  6.4  4.2  6.1  2.4 11.6  1.7 11.
 12.2  4.5  5.5  9.5  4.3  6.7  3.   6.9]


In [33]:
np.mean(murder_data)

np.float64(7.377999999999999)

In [41]:
t_stat = (np.mean(murder_data) - 8.5) /(np.std(murder_data, ddof=1) /np.sqrt(len(murder_data)))

In [35]:
df = len(murder_data)

In [36]:
p_value = 2 * stats.t.sf(np.abs(t_stat),df)

In [39]:
ci= stats.t.interval(0.95, df, 
                        loc =np.mean(murder_data),
                     scale = np.std(murder_data, ddof=1)/np.sqrt(murder_data))

In [42]:
print("Mean:", np.mean(murder_data))
print("sample standard deviation:", np.std(murder_data))

Mean: 7.377999999999999
sample standard deviation: 3.6544378500666825


In [43]:
print("Test Statics: " , t_stat)

Test Statics:  -2.1491677577323403


t_stat = -2.149
This means:
- The sample mean -2.149 standard errors away from 8.5
- Negative sign means the sample mean is below 8.5
- The larger the |t|, the more unusual our sample is

(mcq exam ma aauna sakcha values ko meaning)

|t| = 0.5 ---> Very close to hypothesized mean (not unusual)
|t| = 1.5 ---> Moderately far from hypothesized mean
|t| = 2.1 ---> far form hypthesized mean

In [ ]:
k